---
title: Joint probability and FMC embedding
--- 

In [ ]:
# Always import phasic first to set jax backend correctly
import phasic
import numpy as np
np.random.seed(42)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('retina', 'png')
import matplotlib
matplotlib.rcParams['figure.figsize'] = (5, 3.7)
sns.set_context('paper', font_scale=0.9)
phasic.set_theme('dark')

## Example model: Coalescent

In [ ]:

def coalescent(state, nr_samples=None):
    if not state.size:
        ipv = [([nr_samples]+[0]*nr_samples, 1)]
        return ipv
    else:
        transitions = []
        for i in range(nr_samples):
            for j in range(i, nr_samples):            
                same = int(i == j)
                if same and state[i] < 2:
                    continue
                if not same and (state[i] < 1 or state[j] < 1):
                    continue 
                new = state.copy()
                new[i] -= 1
                new[j] -= 1
                new[i+j+1] += 1
                transitions.append((new, state[i]*(state[j]-same)/(1+same)))
        return transitions

graph = Graph(callback=coalescent, nr_samples=4)

graph.plot()

In [ ]:
graph.expectation()

In [ ]:
rewards = graph.states().T
rewards

In [ ]:
sfs = np.apply_along_axis(graph.expectation, 1, rewards)
sns.barplot(sfs) ;

## Make discrete

Turn state space for a continuous PTD into one for a discrete.

In [ ]:
def make_discrete(graph, mutation_rate, skip_states=[], skip_slots=[]):
    """
    Takes a graph for a continuous distribution and turns
    it into a descrete one (inplace). Returns a matrix of
    rewards for computing marginal moments
    """

    mutation_graph = graph.copy()

    # save current nr of states in graph
    vlength = mutation_graph.vertices_length()

    # number of fields in state vector (assumes all are the same length)
    state_vector_length = len(mutation_graph.vertex_at(1).state())

    # list state vector fields to reward at each auxiliary node
    # rewarded_state_vector_indexes = [[] for _ in range(state_vector_length)]
    rewarded_state_vector_indexes = defaultdict(list)

    # loop all but starting node
    for i in range(1, vlength):
        if i in skip_states:
            continue
        vertex = mutation_graph.vertex_at(i)
        if vertex.rate() > 0: # not absorbing
            for j in range(state_vector_length):
                if j in skip_slots:
                    continue
                val = vertex.state()[j]
                if val > 0: # only ones we may reward
                    # add auxilliary node
                    mutation_vertex = mutation_graph.create_vertex(np.repeat(0, state_vector_length))
                    mutation_vertex.add_edge(vertex, 1)
                    vertex.add_edge(mutation_vertex, mutation_rate*val)
                    # print(mutation_vertex.index(), rewarded_state_vector_indexes[j], j)
                    # rewarded_state_vector_indexes[mutation_vertex.index()] = rewarded_state_vector_indexes[j] + [j]
                    rewarded_state_vector_indexes[mutation_vertex.index()].append(j)

    # print(rewarded_state_vector_indexes)

    # normalize graph
    weights_were_multiplied_with = mutation_graph.normalize()

    # build reward matrix
    rewards = np.zeros((mutation_graph.vertices_length(), state_vector_length))
    for state in rewarded_state_vector_indexes:
        for i in rewarded_state_vector_indexes[state]:
            rewards[state, i] = 1

    rewards = np.transpose(rewards)
    return mutation_graph, rewards



graph = Graph(callback=coalescent, nr_samples=3)

# self-transition rate:
# mutation_rate = 1e-8
mutation_rate = 0.1

# # clone graph to get one to modify:
# mutation_graph = graph.copy()

# add auxilliary states, normalize and return reward matrix:
mutation_graph, rewards = make_discrete(graph, mutation_rate)

# print(mutation_graph.expectation())
# print([mutation_graph.expectation(r) for r in rewards])

# print(rewards)
mutation_graph.plot()


# from functools import wraps

# def discrete(mutation_rate, skip_states=[], skip_slots=[]):
#     def decorator(graph_constructor):
#         @wraps(graph_constructor)
#         def wrapper(*args, **kwargs):
#             graph = graph_constructor(*args, **kwargs)
#             rewards = make_discrete(graph, mutation_rate)
#             return rewards
#         return wrapper
#     return decorator
    
# @discrete(mutation_rate=1)
# def foo():    
#     return coalescent(4)


# rewards = foo()
# rewards

In [ ]:
discrete_graph, discrete_rewards= graph.discretize(reward_rate=0.1)

In [ ]:
mutation_graph.states()

In [ ]:
rewards

In [ ]:
graph.expectation() * 0.1


In [ ]:
graph.expectation(np.sum(mutation_graph.states(), axis=0) * 0.1), 


In [ ]:
sfs = np.apply_along_axis(mutation_graph.expectation_discrete, 1, rewards)
sns.barplot(sfs) ;

In [ ]:
sns.barplot(mutation_graph.pdf(np.arange(10)))
sns.despine()

## Discrete joint prob

In [ ]:

def discrete_joint_prob(graph, reward_rates, precision=1e-15, return_fun=False, return_graph=False):

    starting_vertex = graph.starting_vertex()
    reward_dims = len(reward_rates(starting_vertex.state())) - 1 # a bit of a hack. -1 to not count trash rate...

    orig_state_vector_length = len(graph.vertex_at(1).state())
    state_vector_length = orig_state_vector_length + reward_dims

    state_indices = np.arange(orig_state_vector_length)
    reward_indices = np.arange(orig_state_vector_length, state_vector_length)

    new_graph = Graph(state_vector_length)
    # new_starting_vertex = new_graph.vertex_at(1)
    new_starting_vertex = new_graph.starting_vertex()

    null_rewards = np.zeros(reward_dims)

    index = 0
    # add edges from starting vertex (IPV)
    for edge in starting_vertex.edges():
        new_starting_vertex.add_edge(
          new_graph.find_or_create_vertex(np.append(edge.to().state(), null_rewards).astype(int)), 1)

    index = index + 1
    
    trash_rates = {}
    t_vertex_indices = np.array([], dtype=int)
    while index < new_graph.vertices_length():

        new_vertex = new_graph.vertex_at(index)
        new_state = new_vertex.state()
        state = new_vertex.state()[state_indices]
        vertex = graph.find_vertex(state)

        # non-mutation transitions (coalescence)
        for edge in vertex.edges():
            new_child_state = np.append(edge.to().state(), new_state[reward_indices])

            if np.all(new_state == new_child_state):
                continue
                
            new_child_vertex = new_graph.find_or_create_vertex(new_child_state)
            # cat(new_child_vertex$state, "\n")
            new_vertex.add_edge(new_child_vertex, # if I use create_vertex here, I cannot find it again with find_vertex...
                edge.weight()
            )

            # if new child was absorbing, record at "t-states":
            if not graph.find_vertex(new_child_state[state_indices]).edges():
                t_vertex_indices = np.append(t_vertex_indices, new_child_vertex.index()) 

        # mutation transitions
        current_state = new_state[state_indices]
        current_rewards = new_state[reward_indices]
        rates = reward_rates(current_state, current_rewards) # list of all allowed mutation transition rates with trash rate appended

        trash_rates[index] = rates[reward_dims]
        for i in range(reward_dims):
            rate = rates[i]
            if rate > 0:
                new_rewards = current_rewards
                new_rewards[i] = new_rewards[i] + 1
                new_child_vertex = new_graph.find_or_create_vertex(np.append(current_state, new_rewards))
                # stopifnot(sum(new_child_vertex$state) > 4)
                # cat(new_child_vertex$state, "\n")
                new_vertex.add_edge(
                    new_child_vertex, # if I use create_vertex here, I cannot find it again with find_vertex...
                    rate
                    )
                
                # # if new child was absorbing, record at "t-states":                
                # if (length(edges(find_vertex(graph, new_child_state[state_indices]))) == 0) {
                #     t_vertex_indices = c(t_vertex_indices, new_child_vertex$index) 

        index = index + 1 

        if not index % 10_000:
            graph_size = new_graph.vertices_length()
            print(f'index: {index:>6}      vertices: {graph_size:>6}      ratio: {graph_size/index:>4.2}', file=sys.stderr)
            sys.stderr.flush()

    # trash states
    trash_vertex = new_graph.find_or_create_vertex(np.repeat(0, state_vector_length))
    trash_loop_vertex = new_graph.create_vertex(np.repeat(0, state_vector_length))
    trash_vertex.add_edge(trash_loop_vertex, 1)
    trash_loop_vertex.add_edge(trash_vertex, 1)

    # add trash edges
    for i, rate in trash_rates.items():
        new_graph.vertex_at(i).add_edge(trash_vertex, rate) 

    # add edges from t-states to new final absorbing
    new_absorbing = new_graph.create_vertex(np.repeat(0, state_vector_length))

    t_vertex_indices = np.unique(t_vertex_indices)
    
    for i in t_vertex_indices:
        new_graph.vertex_at(i).add_edge(new_absorbing, 1)

    # normalize graph                            
    weights_were_multiplied_with = new_graph.normalize()

    if return_graph:                           
        return(new_graph)                                             

    # time spent in each of the the t-states at time stop or after some appropriately large time (these are the joint probs)

    prev = None
    for decade in range(1000):
        accum_time_all = new_graph.accumulated_visiting_time(decade*10)
        accum_time = np.array(accum_time_all)[t_vertex_indices]
        if prev is not None and np.all(np.abs(accum_time - prev) < precision):
            break
        prev = accum_time

    assert decade < 100

    class Fun():

        def __init__(self, new_graph, t_vertex_indices):
            self.new_graph = new_graph
            self.t_vertex_indices = t_vertex_indices

        def __call__(self, tup):

        # def __call__(self, stop):
        #     accum_time_all = self.new_graph.accumulated_visiting_time(stop)
        #     accum_time = np.array(accum_time_all)[self.t_vertex_indices]

        #     states = new_graph.states()
        #     state_reward_matrix = states[self.t_vertex_indices, :][:, reward_indices]
        #     joint_probs = pd.DataFrame(state_reward_matrix)
        #     index_cols = joint_probs.columns.values.tolist()
        #     joint_probs['time'] = stop
        #     joint_probs['prob'] = accum_time
        #     joint_probs.set_index(index_cols, inplace=True)

            return joint_probs 

    fun = Fun(new_graph, t_vertex_indices)

    if return_fun:
        return fun

    return fun(decade*10).drop(columns='time')

    # I can test if the graph is acyclic and if so, use accumulated_residence_time instead?


In [ ]:
graph = Graph(callback=coalescent, nr_samples=4)
graph.plot()

In [ ]:
graph.variance()

In [ ]:
# gam = graph.as_matrices()
# gam

In [ ]:
graph.plot()

In [ ]:
graph.pdf(10)

In [ ]:
graph.plot(
            nodesep=0.5,
             subgraphfun=lambda state: 'has singletons' if np.all(state[0] > 0) else 'no singletons',
               )

In [ ]:
def reward_callback(state, current_rewards=None, mutation_rate=1, reward_limit=10, tot_reward_limit=np.inf):

    reward_limits = np.append(np.repeat(reward_limit, len(state)-1), 0)
    
    reward_dims = len(reward_limits)
    if current_rewards is None:
        current_rewards = np.zeros(reward_dims)

    reward_rates = np.zeros(reward_dims)
    trash_rate = 0
    
    for i in range(reward_dims):
        rate = state[i] * mutation_rate 
        r = np.zeros(reward_dims)
        r[i] = 1
        if np.all(current_rewards + r <= reward_limits) and np.sum(current_rewards + r) <= tot_reward_limit:
            reward_rates[i] = rate
        else:
            trash_rate = trash_rate + rate

    return np.append(reward_rates, trash_rate)

joint_probs = discrete_joint_prob(graph, reward_callback)

joint_probs    

In [ ]:
def reward_callback(state, current_rewards=None, mutation_rate=1, reward_limit=10, tot_reward_limit=np.inf):

    reward_limits = np.append(np.repeat(reward_limit, len(state)-1), 0)
    
    reward_dims = len(reward_limits)
    if current_rewards is None:
        current_rewards = np.zeros(reward_dims)

    reward_rates = np.zeros(reward_dims)
    trash_rate = 0
    
    for i in range(reward_dims):
        rate = state[i] * mutation_rate 
        r = np.zeros(reward_dims)
        r[i] = 1
        if np.all(current_rewards + r <= reward_limits) and np.sum(current_rewards + r) <= tot_reward_limit:
            reward_rates[i] = rate
        else:
            trash_rate = trash_rate + rate

    return np.append(reward_rates, trash_rate)


def joint_pmf(graph, rate_fun, reward_limit=10):
    """
    Returns a joint probability mass function for the graph
    """

    def reward_callback(state, current_rewards=None, mutation_rate=1, reward_limit=10):

        reward_limits = np.append(np.repeat(reward_limit, len(state)-1), 0)
        
        reward_dims = len(reward_limits)
        if current_rewards is None:
            current_rewards = np.zeros(reward_dims)

        reward_rates = np.zeros(reward_dims)
        trash_rate = 0
        
        for i in range(reward_dims):
            rate = rate_fun(state[i])
            r = np.zeros(reward_dims)
            r[i] = 1
            if np.all(current_rewards + r <= reward_limits):
                reward_rates[i] = rate
            else:
                trash_rate = trash_rate + rate

        return np.append(reward_rates, trash_rate)

    return discrete_joint_prob(graph, reward_callback, return_fun=True)



fun = joint_pmf(graph, lambda x: x*1, reward_limit=10)
fun

In [ ]:
joint_probs.at[(0, 1, 0, 0, 0), 'prob']

In [ ]:
def joint_pmf(graph, reward_callback):
    df = 
    df.loc[(0, 1, 0, 0)]


In [ ]:
outcomes = np.matrix(list(map(list, joint_probs.index.values)))
probs = joint_probs['prob'].values
with_deficit = probs @ outcomes
with_deficit = with_deficit[:,:nr_samples-1]
no_deficit = np.matrix([2/x for x in range(1, 4)])
deficit = (no_deficit - with_deficit) / no_deficit
deficit

In [ ]:
joint_prob_at_time = discrete_joint_prob(graph, reward_rates, return_fun=True)
joint_prob_at_time

In [ ]:
df = pd.concat([joint_prob_at_time(t) for t in np.arange(1, 10, 1)])
df.head(20)

In [ ]:
df.pivot(columns='time')

In [ ]:
new_graph = discrete_joint_prob(graph, reward_rates, return_graph=True)

In [ ]:
new_graph.plot(size=(8, 8), ranksep=0.6, nodesep=0.3, rainbow=True)

In [ ]:

new_graph.plot(size=(8, 8), ranksep=3, nodesep=0.3, rainbow=True,
    subgraphfun=lambda state: ','.join(map(str, state[:nr_samples])),
    splines='line',
)

## Finite Markov Chains (FMC)

### Distribution of steps spent in a state

In [ ]:
def loop():

    def callback(state):
        if not state.size:
            return [([1, 0], 0.5), ([0, 1], 0.5)]

        transitions = []

        if state.sum() > 1:
            return transitions

        if state[0] == 0:
            new_state = state.copy()
            new_state[0] += 1
            new_state[1] -= 1            
            transitions.append((new_state, 1))
        else:
            new_state = state.copy()
            new_state[0] -= 1
            new_state[1] += 1
            transitions.append((new_state, 1))

        new_state = state.copy()
        new_state[0] = 9
        new_state[1] = 9
        transitions.append((new_state, 1))

        return transitions

    graph = Graph(callback=callback)
    return graph
        
graph = loop()
graph.plot(rainbow=True)


In [ ]:

def loop_reward_rates(new_state, current_rewards=None, mutation_rate=None, reward_limit=10, tot_reward_limit=5):

    target_state = np.array([0, 1])

    reward_limits = np.append(np.repeat(reward_limit, len(new_state)-1), 0)
    
    reward_dims = len(reward_limits)
    if current_rewards is None:
        current_rewards = np.zeros(reward_dims)

    result = np.zeros(reward_dims)
    trash_rate = 0
    
    for i in range(reward_dims):
        rate = new_state[i] * mutation_rate 
        r = np.zeros(reward_dims)
        r[i] = 1
        
        if np.all(new_state == target_state) and np.sum(current_rewards + r) <= tot_reward_limit:
        # if np.all(current_rewards + r <= reward_limits) and np.sum(current_rewards + r) <= tot_reward_limit:
            result[i] = rate
        else:
            trash_rate = trash_rate + rate

    return np.append(result, trash_rate)


# reward_rates = partial(coalescent_reward_rates, mutation_rate=1, reward_limit=1, tot_reward_limit=2)
reward_rates = partial(loop_reward_rates, mutation_rate=1, reward_limit=2)

joint_probs = discrete_joint_prob(graph, reward_rates)

joint_probs#.head()

In [ ]:
discrete_joint_prob(graph, reward_rates, return_graph=True).plot(size=(8, 8), ranksep=1, nodesep=0.5)

### Nr of runs of a particular state

In [ ]:
sample_size = 5
graph = coalescent(sample_size)
graph.plot(rainbow=True)


In [ ]:



new_graph = graph.copy()

target_vertex = new_graph.vertex_at(5)

new_vertices = []
for vertex in new_graph.vertices():
    for edge in vertex.edges():
        if edge.to() == target_vertex:
            # create new vertex
            new_vertex = new_graph.find_or_create_vertex(np.repeat(-vertex.index(), vertex.state().size))

            # make edge point that instead
            edge.update_to(new_vertex)

            # add edge from new_vertex to target_vertex with weight 1
            new_vertex.add_edge(target_vertex, 1)

            # keep index of added vertex
            new_vertices.append(new_vertex.index())

# rewards = make_discrete(new_graph, mutation_rate, skip=new_vertices)
rewards = make_discrete(new_graph, 1, skip_states=new_vertices)

print(rewards)

rev = np.zeros(new_graph.vertices_length())
for i in new_vertices:
    rev[i] = 1

print(new_graph.expectation(rev))
# print(new_graph.accumulated_visiting_time(10))
print(new_graph.accumulated_visits_discrete(1000))

new_graph.plot(rainbow=True, size=(10, 10))


In [ ]:
def loop(state):
    if not state.size:
        return [([1], 1)]
    elif state[0] < 3:
        return [(state+1, 4)]
    return []

graph = ptd.Graph(callback=loop)
graph.plot()

In [ ]:
graph.expectation(), graph.expected_waiting_time()

In [ ]:

discr_graph, discr_rewards = graph.discretize(reward_rate=1)
discr_graph.plot()


In [ ]:
np.sum(discr_rewards, axis=0)

In [ ]:
discr_graph.expected_waiting_time()

In [ ]:
discr_graph.expectation(np.sum(discr_rewards, axis=0))

In [ ]:
np.sum(discr_rewards, axis=0)

In [ ]:
discr_graph.expectation(discr_rewards)


In [ ]:
graph.expectation(np.sum(discr_rewards, axis=1))

In [ ]:
discr_graph.expectation(rewards = np.sum(discr_rewards, axis=1))
#graph.expectation(np.sum(discr_rewards, axis=1) * 0.1)




In [ ]:
def loop():

    def callback(state):
        transitions = []
        if state[0] == 0:
            new_state = state[:]
            new_state[0] += 1
            new_state[1] -= 1
            transitions.append((new_state, 1))
        else:
            new_state = state[:]
            new_state[0] -= 1
            new_state[1] += 1
            transitions.append((new_state, 1))
        return transitions

    graph = ptd.Graph(callback=callback, initial=[1, 0])
    return graph
        
graph = loop()
graph.plot(rainbow=True)

### I could add an fmc argument that adds a slot to initial and increments that for all states returned by callback

In [ ]:
def loop():

    def callback(state):

        if not state.size:
            return [([1, 0, 0], 1)]

        if state[2] == 5:
            return []

        if state[0] == 0:
            new_state = state[:]
            new_state[0] += 1
            new_state[1] -= 1
            new_state[2] += 1
            return [(new_state, 1)]
        else:
            new_state = state[:]
            new_state[0] -= 1
            new_state[1] += 1
            new_state[2] += 1
            return [(new_state, 1)]

    graph = ptd.Graph(callback=callback)
    return graph
        
graph = loop()
graph.plot(rainbow=True)

In [ ]:
rewards = make_discrete(graph, 1, skip_slots=[graph.state_length()-1])
graph.plot(rainbow=True)


In [ ]:
graph.states()

In [ ]:
graph.accumulated_visits_discrete(1000)


### Length of longest run of a state in an FMC

In [ ]:
# D: A->A
# A: A->B
# B: B->A
# D: B->B

def maxlen():

    def callback(state):

        A, B, C, D = 1, 1, 1, 1

        if not state.size:
            return [([1, 0, 0], 1)]

        x, y, z = state
        
        max_y, max_z = 2, 6

        absorb = (0, max_y, max_z)
        # absorb = (0, 0, 0)
        if np.all(state == absorb):
            return []

        trash1, trash2 = (-1, -1, -1), (-2, -2, -2)
        # trash1, trash2 = (0, 0, 0), (0, 0, 0)
        if np.all(state == trash1):
            return [(trash2, 1)]
        if np.all(state == trash2):
            return [(trash1, 1)]

        if z+1 == max_z:
            return [(absorb, 1)]
        
        if y == 0:
            return [((x, y, z+1), C/(B+C)),
                    ((x, y+1, z+1), A/(A+D))]
        if y == max_y:
            return [(trash1, D/(A+D)), # trash
                    ((y, 0, z+1), B/(B+C))]

        return [((x, y+1, z+1), D/(A+D)),
                ((y, 0, z+1), B/(B+C))]

    graph = ptd.Graph(callback=callback)
    return graph
        
graph = maxlen()
graph.plot(rainbow=True, ranksep=1, nodesep=0.3, size=(10, 10))

In [ ]:
graph.states()

In [ ]:
joint_probs %>% filter(V1==1, V2==0, V3==1) 

With `sample_size <- 4`, `mutation_rate <- 1`, `reward_limits <- rep(20, sample_size-1)`, and `tot_reward_limit <- Inf`, tothe marginal probabilities match the SFS:

```
c(sum(joint_probs$V1 * joint_probs$accum_time), 
  sum(joint_probs$V2 * joint_probs$accum_time), 
  sum(joint_probs$V3 * joint_probs$accum_time))
```

```
1.99981743759578 0.998152771395828 0.666638375487117
```

and the joint prob of a singleton and a trippleton is:

```
1	0	1	0.03111111
```

which is exactly what we also get with `reward_limits <- rep(1, sample_size-1)`.

Setting `tot_reward_limit <- 2` also produces `0.03111111`.

In [ ]:
joint_probs %>% rename_with(gsub, pattern="V", replacement="ton") %>% group_by(ton1, ton2) %>% summarize(prob=sum(accum_time), .groups="keep")

## Two-locus ARG

In [ ]:
ggplot(plot_df, aes(x=ton0x1, y=ton1x0)) +
    geom_tile(aes(fill = accum_time)) + 
    geom_text(aes(label = round(accum_time, 3))) +
    scale_fill_viridis() +
    # scale_fill_distiller(palette = 'PiYG',direction = 1,
    #                 limit=max(abs(plot_df$prob)) * c(-1, 1)
    #                 ) +
    theme_minimal() +
     theme(panel.grid.major = element_blank(), 
            panel.grid.minor = element_blank(), 
            text=element_text(size=17))

# Base-n approach

## State space for joint proability computation

Generate coalescent state space like normal with the following modifications

- Change state space from (4, 0, 0, 0) to (4, 0, 0, 0, t1, t2, t3, t4). The last extra "ton" states keep track of the number accumulated mutations of each kind. We simply double the state vector so we keep track of the counts lineages with descendants, but also the counts of mutations happened on such lineages.
- Each state can mutate to accumulate a "ton" in accordance with its state vector. E.g., a `(4, 0, 0, 0, 0, 0, 0, 0)` state can only make singletons,  a `(2, 1, 0, 0, 0, 0, 0, 0)` state can only make singletons and doubletons.
- A mutation event is a transition to a siter state E.g., `(4, 0, 0, 0, 0, 0, 0, 0) -> (4, 0, 0, 0, 1, 0, 0, 0)`
- The ton counts have a maximum value (base-1). If this value is reached, the mutation transition instead leads to a trash state with an infinite self loop. The transitions to trash represents the part of the deficient PDF not covered because we only run up to a max nr of tons.

## Reward transform

- Convert the last half of each state (with ton counts) to numbers in some base.
- Use these for reward transformation.
- Compute PDF for t <- 1:sample_size^(base-1)
- Convert each time t back to the corresponding ton vector and associate it with the probability
- group by two tons and sum probs in groups to get all pairwise combinations for a joint probability matrix.

## Figure out why you get NAs in multi_rewards with max_tons <- 1

In [ ]:
plot_graph(graph_as_matrix(graph), size=c(10, 8), align=TRUE, # rainbow=TRUE,
               fontsize=16, ranksep=1, nodesep=0.25)

In [ ]:
expectation(graph)

In [ ]:
plot_graph(graph_as_matrix(graph), #rainbow=TRUE, 
           size=c(10, 8), #align=TRUE,
               fontsize=16, ranksep=1, nodesep=0.25,
             # subgraph=TRUE, subgraphfun=function(state, index) as.character((index+1) %/% 2)
)

In [ ]:
apply(rewards[1:3,], 1, function(x) expectation(graph, x))

In [ ]:
pdph(1:10, rev_graph)

## Compute joint prob

> **Constraining the total number of mutations does not work**
> 
> The deficit is computed correctly as long as all max rewards so that all r scalar values in the CDF represents a reward combination in the MDF
> 
> Just like we can limit the number of each king of tons in the state space contruction, we might also limit the total number of mutations so that we, for example, can have at most one instance of two different tons (`total_tons=2`). E.g., a singleton and a tripleton.
> 
> However, this gives a a deficit problem I am not sure I can solve with this approach. In principle, the deficit should be taken care of, and I should just discard all joint probs for total numbers of tons larger than `total_tons` - but that does not seem to be the case...

**maybe I don't need loops if they are not selff-loops anyway. If aux->C has rate 1 then A->aux->C is the same as A->C. Below I just changed two things**

1. normalize the graph
2. use pdph instead of pph

**BUT** if I normalize I need to represent the residual prob as reward, which means I need to reward transform, which I cannot if I want to do everying in one go with the scalar trick. 

In [ ]:
if (vertices_length(graph) < 50)
    plot_graph(graph_as_matrix(graph), size=c(10, 8), align=TRUE, rainbow=TRUE,
               fontsize=16, ranksep=2, nodesep=0.5,
             subgraphs=TRUE,         
           subgraphfun=function(state, index) paste(state[1:sample_size], collapse=""))

Get last halves of states that server as mutation rewards:

Turn reward vectors into scalars (with the appropriate base):

Loop over states except starting to find trash vertices and give them a reward so they won't dissapear in the reward transformation. They will not contribute this reward because they are dead ends:

In [ ]:
multi_rewards

Reward transform graph using scalar rewards:

In [ ]:
if (vertices_length(graph) < 50)
    plot_graph(graph_as_matrix(rew_graph),
           rainbow=TRUE,
           size=c(8, 8), 
           align=TRUE,
           fontsize=14, ranksep=1, nodesep=0.5, 
           # subgraphs=TRUE, subgraphfun=function(state, index) as.character((index+1) %/% 2)
           )

Compute CDF assming no mutation count exceeds `max_tons`:

Convert reward scalars back into state vectors representing ton counts:

The deficit is taken care of, so you should discard all joint probs for total numbers of tons larger than `total_tons`:

In [ ]:
df %>% ggplot(aes(x=t, y=cdf)) + 
    geom_bar(stat="identity") +
    labs(x='scaled time', y='probability') + 
    despine + ylim(0, 1)

Compute probability of standing in on of the trash states for each time t in our CDF. These represent the deficit of the computed CDF:

> Make sure the stop_probability is the discrete version of that is what we are doing

CDF deficit:

In [ ]:
df %>% ggplot(aes(x=t, y=cdf_deficit)) + 
    geom_bar(stat="identity") +
    labs(x='scaled time', y='probability') + 
    despine + ylim(0, 1)

Sanity check: adding CDF and deficit should produce a CDF that goes to 1:

In [ ]:
df %>% ggplot(aes(x=t, y=cdf_incl_deficit)) + 
    geom_bar(stat="identity") +
    labs(x='scaled time', y='probability') + 
    despine + 
    ylim(0, 1) + 
    geom_hline(yintercept=1, linetype="dashed")

I.e., and a PDF that sum to one:

In [ ]:
df %>% ggplot(aes(x=t, y=pdf_from_cdf_incl_deficit)) + 
    geom_bar(stat="identity") +
    labs(x='scaled time', y='probability') + 
    despine

In [ ]:
sum(df$pdf_from_cdf_incl_deficit)

It **almost** does... Maybe a numerical issue

Compute PDF from the CDF (**this is the one we are after**):

In [ ]:
df %>% ggplot(aes(x=t, y=pdf_from_cdf)) + 
    geom_bar(stat="identity") +
    labs(x='scaled time', y='probability') + 
    despine

The reason we need to go through the CDF to get the PDF is that the PDF function in PtD computes the distribution of times when the absorbing state is reached. It this cannot take the deficit in trash_states into account. The PDF commputed directly looks like this:

> Make sure I ues the discrete version here if I also use the dicscrete CDF above

In [ ]:
df %>% ggplot(aes(x=t, y=pdf)) + 
    geom_bar(stat="identity") +
    labs(x='scaled time', y='probability') + 
    despine 

## When we do the discrete version, we don't need to go through the CDF to get the PDF. We can just use the `ddph` directly


The marginal expectations does not match the SFS proportions, because paths that accumulate more than `max_tons` singletons will end in the trash state and not have the opportunity to also accumulate doubletons etc. That reflects that the the joint prob of a singleton *and* a doubleton is be a subset of the singleton probability. That way the total marginal singleton prob will be roughly sfs expectation, but the total marginal doubleton prob will be much too small:

In [ ]:
df[df$X1==1 & df$X2==0 & df$X3==1, ]

In [ ]:
df

In [ ]:
c(sum(joint_probs$V1 * joint_probs$accum_time), 
  sum(joint_probs$V2 * joint_probs$accum_time), 
  sum(joint_probs$V3 * joint_probs$accum_time))

In [ ]:
c(sum(df$X1 * df$prob), sum(df$X2 * df$prob), sum(df$X3 * df$prob))

In [ ]:
ggplot(plot_df, aes(x=X2, y=X3)) +
    geom_tile(aes(fill = prob)) + 
    geom_text(aes(label = round(prob, 3))) +
    scale_fill_distiller(palette = 'PiYG',direction = 1,
                    limit=max(abs(plot_df$prob)) * c(-1, 1)
                    ) +
    theme_minimal() +
     theme(panel.grid.major = element_blank(), 
            panel.grid.minor = element_blank(), 
            text=element_text(size=17))

In [ ]:
ggplot(plot_df, aes(x=X2, y=X3)) +
    geom_tile(aes(fill = log10(prob))) + 
    geom_text(aes(label = round(log10(prob), 2))) +
    scale_fill_distiller(palette = 'PiYG',direction = 1,
                    limit=max(abs(log10(plot_df$prob))) * c(-1, 1)
                    ) +
theme_minimal() +
 theme(panel.grid.major = element_blank(), 
        panel.grid.minor = element_blank(), 
        text=element_text(size=17))
